In [ ]:
"""
This code is my solution to the assignment for the course "Machine Learning" at FAKT Szakkollégium.
The assigment was a Kaggle competition, where the goal was to predict the popularity of articles based on the given
data. I added the data to the repository, so you can check it out.

Description of the task: This is a private binary classification competition for the Introduction to Machine Learning
Tools, at fakt in the Fall semester of 2022/23.

In this competition, your task is to predict which articles are shared the most on social media. The data comes from the
 website mashable.com from the beginning of 2015.
"""

In [129]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from keras.callbacks import EarlyStopping, LearningRateScheduler
from keras.layers import Dense
from keras.metrics import AUC
from keras.models import Sequential
from sklearn.metrics import roc_auc_score

In [132]:
# Load the data
df = pd.read_csv("../Case studies/News popularity/train.csv")
df.head()

,timedelta,n_tokens_title,n_tokens_content,n_unique_tokens,n_non_stop_words,n_non_stop_unique_tokens,num_hrefs,num_self_hrefs,num_imgs,num_videos,...,max_positive_polarity,avg_negative_polarity,min_negative_polarity,max_negative_polarity,title_subjectivity,title_sentiment_polarity,abs_title_subjectivity,abs_title_sentiment_polarity,is_popular,article_id
0,594,9,702,0.454545,1.0,0.620438,11,2,1,0,...,1.000000,-0.153395,-0.4,-0.10,0.0,0.0,0.5,0.0,0,1
1,346,8,1197,0.470143,1.0,0.666209,21,6,2,13,...,1.000000,-0.308167,-1.0,-0.10,0.0,0.0,0.5,0.0,0,3
2,484,9,214,0.618090,1.0,0.748092,5,2,1,0,...,0.433333,-0.141667,-0.2,-0.05,0.0,0.0,0.5,0.0,0,5
3,639,8,249,0.621951,1.0,0.664740,16,5,8,0,...,0.500000,-0.500000,-0.8,-0.40,0.0,0.0,0.5,0.0,0,6
4,177,12,1219,0.397841,1.0,0.583578,21,1,1,2,...,0.800000,-0.441111,-1.0,-0.05,0.0,0.0,0.5,0.0,0,7


In [133]:
df = df.drop('article_id', axis=1).drop('timedelta', axis=1)
columns_to_standardize = ['n_tokens_title', 'n_tokens_content', 'n_unique_tokens', 'n_non_stop_words',
                          'n_non_stop_unique_tokens', 'num_hrefs', 'num_self_hrefs', 'num_imgs', 'num_videos',
                          'average_token_length', 'num_keywords', 'kw_min_min', 'kw_max_min', 'kw_avg_min',
                          'kw_min_max', 'kw_max_max', 'kw_avg_max', 'kw_min_avg', 'kw_max_avg', 'kw_avg_avg',
                          'self_reference_min_shares', 'self_reference_max_shares', 'self_reference_avg_sharess',
                          'LDA_00', 'LDA_01', 'LDA_02', 'LDA_03', 'LDA_04', 'global_subjectivity',
                          'global_sentiment_polarity', 'global_rate_positive_words', 'global_rate_negative_words',
                          'rate_positive_words', 'rate_negative_words', 'avg_positive_polarity',
                          'min_positive_polarity', 'max_positive_polarity', 'avg_negative_polarity',
                          'min_negative_polarity', 'max_negative_polarity', 'title_subjectivity',
                          'title_sentiment_polarity', 'abs_title_subjectivity', 'abs_title_sentiment_polarity']
scaler = StandardScaler()
df[columns_to_standardize] = scaler.fit_transform(df[columns_to_standardize])

In [134]:
x = df.iloc[:, :-1]
y = df.iloc[:, -1]

x,y

(       n_tokens_title  n_tokens_content  n_unique_tokens  n_non_stop_words  \
 0           -0.659121          0.334487        -0.024734         -0.000969   
 1           -1.133033          1.389137        -0.020896         -0.000969   
 2           -0.659121         -0.705248         0.015504         -0.000969   
 3           -1.133033         -0.630677         0.016453         -0.000969   
 4            0.762613          1.436010        -0.038685         -0.000969   
 ...               ...               ...              ...               ...   
 29728       -0.185210         -0.360090        -0.009823         -0.000969   
 29729        0.288701          0.057509        -0.007195         -0.000969   
 29730        1.236524          1.450925        -0.055998         -0.000969   
 29731       -2.080855         -0.566759         0.023354         -0.000969   
 29732       -0.185210         -0.854390         0.037413         -0.000969   
 
        n_non_stop_unique_tokens  num_hrefs  num_s

In [135]:
x_train, x_test, y_train, y_test = train_test_split(
    x, y, test_size=0.25, random_state=77)

In [145]:
#Egy réteggel bővített neurális háló, Adam optimalizálás.
model = Sequential()

model.add(Dense(100, input_dim=x.shape[1], activation='relu',
                kernel_initializer='random_normal'))
model.add(Dense(100, activation='relu',
                kernel_initializer='random_normal'))
model.add(Dense(50,activation='relu',kernel_initializer='random_normal'))
model.add(Dense(50,activation='relu',kernel_initializer='random_normal'))
model.add(Dense(1,activation='sigmoid',kernel_initializer='random_normal'))
model.compile(
    loss='binary_crossentropy', 
    optimizer='adam',
    metrics=[AUC(name='auc'), 'accuracy'])

earlystop = EarlyStopping(monitor='val_loss', min_delta=1e-3, 
 patience=5, verbose=1, mode='auto', restore_best_weights=True)

model.summary()

Model: "sequential_37"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_156 (Dense)           (None, 100)               5900      
                                                                 
 dense_157 (Dense)           (None, 100)               10100     
                                                                 
 dense_158 (Dense)           (None, 50)                5050      
                                                                 
 dense_159 (Dense)           (None, 50)                2550      
                                                                 
 dense_160 (Dense)           (None, 1)                 51        
                                                                 
Total params: 23,651
Trainable params: 23,651
Non-trainable params: 0
_________________________________________________________________


In [146]:
model.fit(x_train, y_train, epochs=100, batch_size=64, validation_data=(x_test, y_test), callbacks=[earlystop])

Epoch 1/100
349/349 [==============================] - 2s 3ms/step - loss: 0.3726 - auc: 0.6509 - accuracy: 0.8753 - val_loss: 0.3520 - val_auc: 0.6629 - val_accuracy: 0.8812
Epoch 2/100
349/349 [==============================] - 1s 2ms/step - loss: 0.3433 - auc: 0.7048 - accuracy: 0.8774 - val_loss: 0.3472 - val_auc: 0.6779 - val_accuracy: 0.8812
Epoch 3/100
349/349 [==============================] - 1s 2ms/step - loss: 0.3393 - auc: 0.7167 - accuracy: 0.8774 - val_loss: 0.3492 - val_auc: 0.6763 - val_accuracy: 0.8812
Epoch 4/100
349/349 [==============================] - 1s 2ms/step - loss: 0.3377 - auc: 0.7208 - accuracy: 0.8774 - val_loss: 0.3480 - val_auc: 0.6804 - val_accuracy: 0.8812
Epoch 5/100
349/349 [==============================] - 1s 3ms/step - loss: 0.3350 - auc: 0.7284 - accuracy: 0.8774 - val_loss: 0.3516 - val_auc: 0.6845 - val_accuracy: 0.8812
Epoch 6/100
349/349 [==============================] - 1s 3ms/step - loss: 0.3326 - auc: 0.7350 - accuracy: 0.8774 - val_loss

In [147]:
model.evaluate(x_test, y_test)

233/233 [==============================] - 0s 1ms/step - loss: 0.3440 - auc: 0.6860 - accuracy: 0.8812


[0.34401607513427734, 0.6860003471374512, 0.8812214136123657]

In [139]:
#Ridge büntető paraméter
model = Sequential()

model.add(Dense(100, activation='relu',input_dim=x.shape[1], kernel_initializer='random_normal', 
    kernel_regularizer='l2'))
model.add(Dense(100, activation='relu', kernel_initializer='random_normal', kernel_regularizer='l2'))
model.add(Dense(50, activation='relu', kernel_initializer='random_normal', kernel_regularizer='l2'))
model.add(Dense(50, activation='relu', kernel_initializer='random_normal', kernel_regularizer='l2'))
model.add(Dense(1, activation='sigmoid', kernel_initializer='random_normal'))


model.compile(loss='binary_crossentropy', optimizer='adam', metrics=[AUC(name='auc'), 'accuracy'])

model.summary()

Model: "sequential_35"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_146 (Dense)           (None, 100)               5900      
                                                                 
 dense_147 (Dense)           (None, 100)               10100     
                                                                 
 dense_148 (Dense)           (None, 50)                5050      
                                                                 
 dense_149 (Dense)           (None, 50)                2550      
                                                                 
 dense_150 (Dense)           (None, 1)                 51        
                                                                 
Total params: 23,651
Trainable params: 23,651
Non-trainable params: 0
_________________________________________________________________


In [140]:
model.fit(x_train, y_train, epochs=100, batch_size=64, validation_data=(x_test, y_test), callbacks=[earlystop])

Epoch 1/100
349/349 [==============================] - 2s 3ms/step - loss: 0.4826 - auc: 0.6226 - accuracy: 0.8752 - val_loss: 0.3758 - val_auc: 0.6649 - val_accuracy: 0.8812
Epoch 2/100
349/349 [==============================] - 1s 2ms/step - loss: 0.3775 - auc: 0.6754 - accuracy: 0.8774 - val_loss: 0.3720 - val_auc: 0.6592 - val_accuracy: 0.8812
Epoch 3/100
349/349 [==============================] - 1s 2ms/step - loss: 0.3754 - auc: 0.6667 - accuracy: 0.8774 - val_loss: 0.3703 - val_auc: 0.6575 - val_accuracy: 0.8812
Epoch 4/100
349/349 [==============================] - 1s 3ms/step - loss: 0.3745 - auc: 0.6633 - accuracy: 0.8774 - val_loss: 0.3678 - val_auc: 0.6664 - val_accuracy: 0.8812
Epoch 5/100
349/349 [==============================] - 1s 2ms/step - loss: 0.3730 - auc: 0.6699 - accuracy: 0.8774 - val_loss: 0.3679 - val_auc: 0.6635 - val_accuracy: 0.8812
Epoch 6/100
349/349 [==============================] - 1s 3ms/step - loss: 0.3717 - auc: 0.6745 - accuracy: 0.8774 - val_loss

In [141]:
model.evaluate(x_test, y_test)

233/233 [==============================] - 0s 2ms/step - loss: 0.3638 - auc: 0.6632 - accuracy: 0.8812


[0.36378952860832214, 0.6632379293441772, 0.8812214136123657]

In [148]:
# Trying SGD optimizer
model = Sequential()

model.add(Dense(100, input_dim=x.shape[1], activation='relu',
                kernel_initializer='random_normal',
                kernel_regularizer='l2'))
model.add(Dense(100, activation='relu',
                kernel_initializer='random_normal', kernel_regularizer='l2'))
model.add(Dense(50, activation='relu', kernel_initializer='random_normal', kernel_regularizer='l2'))
model.add(Dense(50, activation='relu', kernel_initializer='random_normal', kernel_regularizer='l2'))
model.add(Dense(1, activation='sigmoid', kernel_initializer='random_normal', kernel_regularizer='l2'))
model.compile(
    loss='binary_crossentropy',
    optimizer='sgd',
    metrics=[AUC(name='auc'), 'accuracy']
)

model.summary()

Model: "sequential_38"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_161 (Dense)           (None, 100)               5900      
                                                                 
 dense_162 (Dense)           (None, 100)               10100     
                                                                 
 dense_163 (Dense)           (None, 50)                5050      
                                                                 
 dense_164 (Dense)           (None, 50)                2550      
                                                                 
 dense_165 (Dense)           (None, 1)                 51        
                                                                 
Total params: 23,651
Trainable params: 23,651
Non-trainable params: 0
_________________________________________________________________


In [149]:
model.fit(x_train, y_train, epochs=100, batch_size=64, validation_data=(x_test, y_test), callbacks=[earlystop])

Epoch 1/100
349/349 [==============================] - 2s 3ms/step - loss: 1.0760 - auc: 0.5003 - accuracy: 0.8758 - val_loss: 0.9439 - val_auc: 0.4991 - val_accuracy: 0.8812
Epoch 2/100
349/349 [==============================] - 1s 2ms/step - loss: 0.8824 - auc: 0.4984 - accuracy: 0.8774 - val_loss: 0.8249 - val_auc: 0.4994 - val_accuracy: 0.8812
Epoch 3/100
349/349 [==============================] - 1s 2ms/step - loss: 0.7936 - auc: 0.5012 - accuracy: 0.8774 - val_loss: 0.7542 - val_auc: 0.4994 - val_accuracy: 0.8812
Epoch 4/100
349/349 [==============================] - 1s 2ms/step - loss: 0.7333 - auc: 0.4919 - accuracy: 0.8774 - val_loss: 0.7007 - val_auc: 0.4994 - val_accuracy: 0.8812
Epoch 5/100
349/349 [==============================] - 1s 2ms/step - loss: 0.6849 - auc: 0.4909 - accuracy: 0.8774 - val_loss: 0.6562 - val_auc: 0.5014 - val_accuracy: 0.8812
Epoch 6/100
349/349 [==============================] - 1s 2ms/step - loss: 0.6439 - auc: 0.4932 - accuracy: 0.8774 - val_loss

In [150]:
model.evaluate(x_test, y_test)

233/233 [==============================] - 0s 1ms/step - loss: 0.3661 - auc: 0.5000 - accuracy: 0.8812


[0.36607858538627625, 0.5, 0.8812214136123657]

In [151]:
# accuracy is the same for all models, but the AUC is the highest for the model with the Adam optimizer

In [152]:
# Dynamic learning rate
def lr_schedule(epoch):
    initial_lr = 0.01
    drop = 0.5
    epochs_drop = 5
    lr = initial_lr * (drop ** (epoch // epochs_drop))
    return float(lr)
lr_scheduler = LearningRateScheduler(lr_schedule)

In [153]:
model = Sequential()

model.add(Dense(100, input_dim=x.shape[1], activation='relu',
                kernel_initializer='random_normal'))
model.add(Dense(100, activation='relu',
                kernel_initializer='random_normal'))
model.add(Dense(50, activation='relu', kernel_initializer='random_normal'))
model.add(Dense(50, activation='relu', kernel_initializer='random_normal'))
model.add(Dense(1, activation='sigmoid', kernel_initializer='random_normal'))

model.compile(
    loss='binary_crossentropy',
    optimizer='adam',
    metrics=[AUC(name='auc'), 'accuracy']
)

model.summary()

Model: "sequential_39"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_166 (Dense)           (None, 100)               5900      
                                                                 
 dense_167 (Dense)           (None, 100)               10100     
                                                                 
 dense_168 (Dense)           (None, 50)                5050      
                                                                 
 dense_169 (Dense)           (None, 50)                2550      
                                                                 
 dense_170 (Dense)           (None, 1)                 51        
                                                                 
Total params: 23,651
Trainable params: 23,651
Non-trainable params: 0
_________________________________________________________________


In [154]:
model.fit(x_train, y_train, epochs=100, batch_size=64, validation_data=(x_test, y_test), callbacks=[earlystop, lr_scheduler])

Epoch 1/100
349/349 [==============================] - 7s 3ms/step - loss: 0.3585 - auc: 0.6701 - accuracy: 0.8773 - val_loss: 0.3503 - val_auc: 0.6772 - val_accuracy: 0.8812 - lr: 0.0100
Epoch 2/100
349/349 [==============================] - 1s 2ms/step - loss: 0.3462 - auc: 0.6994 - accuracy: 0.8774 - val_loss: 0.3491 - val_auc: 0.6839 - val_accuracy: 0.8812 - lr: 0.0100
Epoch 3/100
349/349 [==============================] - 1s 2ms/step - loss: 0.3426 - auc: 0.7082 - accuracy: 0.8774 - val_loss: 0.3431 - val_auc: 0.6861 - val_accuracy: 0.8812 - lr: 0.0100
Epoch 4/100
349/349 [==============================] - 1s 2ms/step - loss: 0.3414 - auc: 0.7124 - accuracy: 0.8774 - val_loss: 0.3546 - val_auc: 0.6774 - val_accuracy: 0.8812 - lr: 0.0100
Epoch 5/100
349/349 [==============================] - 1s 2ms/step - loss: 0.3382 - auc: 0.7225 - accuracy: 0.8774 - val_loss: 0.3480 - val_auc: 0.6757 - val_accuracy: 0.8812 - lr: 0.0100
Epoch 6/100
349/349 [==============================] - 1s 2m

In [155]:
model.evaluate(x_test, y_test)

233/233 [==============================] - 0s 1ms/step - loss: 0.3431 - auc: 0.6861 - accuracy: 0.8812


[0.34309011697769165, 0.6861143708229065, 0.8812214136123657]

In [156]:
# Got better AUC with dynamic learning rate

In [157]:
# Early stopping hyperparameter tuning
min_delta_values = [1e-3, 1e-4, 1e-5, 1e-6]

for min_delta in min_delta_values:
    monitor = EarlyStopping(monitor='val_loss', min_delta=min_delta, patience=5, verbose=1, mode='auto', restore_best_weights=True)
    model.fit(x_train, y_train, validation_data=(x_test, y_test), callbacks=[monitor], verbose=2, epochs=1000)
    test_predictions = model.predict(x_test)
    test_auc = roc_auc_score(y_test, test_predictions)

    print(f'Min Delta: {min_delta}, Test AUC: {test_auc}')



Epoch 1/1000
697/697 - 1s - loss: 0.3396 - auc: 0.7173 - accuracy: 0.8774 - val_loss: 0.3458 - val_auc: 0.6804 - val_accuracy: 0.8812 - 1s/epoch - 2ms/step
Epoch 2/1000
697/697 - 1s - loss: 0.3362 - auc: 0.7278 - accuracy: 0.8773 - val_loss: 0.3453 - val_auc: 0.6820 - val_accuracy: 0.8812 - 1s/epoch - 2ms/step
Epoch 3/1000
697/697 - 1s - loss: 0.3338 - auc: 0.7332 - accuracy: 0.8771 - val_loss: 0.3457 - val_auc: 0.6814 - val_accuracy: 0.8803 - 1s/epoch - 1ms/step
Epoch 4/1000
697/697 - 1s - loss: 0.3322 - auc: 0.7394 - accuracy: 0.8772 - val_loss: 0.3496 - val_auc: 0.6823 - val_accuracy: 0.8808 - 1s/epoch - 1ms/step
Epoch 5/1000
697/697 - 1s - loss: 0.3282 - auc: 0.7483 - accuracy: 0.8776 - val_loss: 0.3506 - val_auc: 0.6727 - val_accuracy: 0.8796 - 1s/epoch - 2ms/step
Epoch 6/1000
Restoring model weights from the end of the best epoch: 1.
697/697 - 1s - loss: 0.3254 - auc: 0.7542 - accuracy: 0.8785 - val_loss: 0.3477 - val_auc: 0.6821 - val_accuracy: 0.8789 - 1s/epoch - 1ms/step
Epoch

In [161]:
# Final model
model = Sequential()

model.add(Dense(100, input_dim=x.shape[1], activation='relu',
                kernel_initializer='random_normal'))
model.add(Dense(100, activation='relu',
                kernel_initializer='random_normal'))
model.add(Dense(50, activation='relu', kernel_initializer='random_normal'))
model.add(Dense(50, activation='relu', kernel_initializer='random_normal'))
model.add(Dense(1, activation='sigmoid', kernel_initializer='random_normal'))

model.compile(
    loss='binary_crossentropy',
    optimizer='adam',
    metrics=[AUC(name='auc'), 'accuracy']
)

earlystop = EarlyStopping(monitor='val_loss', min_delta=1e-5, patience=5, verbose=1, mode='auto', 
                          restore_best_weights=True)

model.summary()

Model: "sequential_41"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_176 (Dense)           (None, 100)               5900      
                                                                 
 dense_177 (Dense)           (None, 100)               10100     
                                                                 
 dense_178 (Dense)           (None, 50)                5050      
                                                                 
 dense_179 (Dense)           (None, 50)                2550      
                                                                 
 dense_180 (Dense)           (None, 1)                 51        
                                                                 
Total params: 23,651
Trainable params: 23,651
Non-trainable params: 0
_________________________________________________________________


In [162]:
model.fit(x_train, y_train, epochs=100, batch_size=64, validation_data=(x_test, y_test), callbacks=[earlystop, lr_scheduler])

Epoch 1/100
349/349 [==============================] - 2s 3ms/step - loss: 0.3604 - auc: 0.6656 - accuracy: 0.8752 - val_loss: 0.3495 - val_auc: 0.6697 - val_accuracy: 0.8812 - lr: 0.0100
Epoch 2/100
349/349 [==============================] - 1s 2ms/step - loss: 0.3482 - auc: 0.6921 - accuracy: 0.8774 - val_loss: 0.3476 - val_auc: 0.6756 - val_accuracy: 0.8812 - lr: 0.0100
Epoch 3/100
349/349 [==============================] - 1s 2ms/step - loss: 0.3439 - auc: 0.7056 - accuracy: 0.8774 - val_loss: 0.3458 - val_auc: 0.6791 - val_accuracy: 0.8812 - lr: 0.0100
Epoch 4/100
349/349 [==============================] - 1s 2ms/step - loss: 0.3413 - auc: 0.7125 - accuracy: 0.8774 - val_loss: 0.3483 - val_auc: 0.6709 - val_accuracy: 0.8812 - lr: 0.0100
Epoch 5/100
349/349 [==============================] - 1s 2ms/step - loss: 0.3411 - auc: 0.7131 - accuracy: 0.8773 - val_loss: 0.3446 - val_auc: 0.6789 - val_accuracy: 0.8812 - lr: 0.0100
Epoch 6/100
349/349 [==============================] - 1s 2m

In [163]:
model.evaluate(x_test, y_test)

233/233 [==============================] - 0s 1ms/step - loss: 0.3446 - auc: 0.6789 - accuracy: 0.8812


[0.3445519506931305, 0.6788502335548401, 0.8812214136123657]

In [164]:
# Importing the test data
test_df = pd.read_csv("../ML/Fakt kurzus/Beadando/test.csv")
test_df.head()

,timedelta,n_tokens_title,n_tokens_content,n_unique_tokens,n_non_stop_words,n_non_stop_unique_tokens,num_hrefs,num_self_hrefs,num_imgs,num_videos,...,min_positive_polarity,max_positive_polarity,avg_negative_polarity,min_negative_polarity,max_negative_polarity,title_subjectivity,title_sentiment_polarity,abs_title_subjectivity,abs_title_sentiment_polarity,article_id
0,134,11,217,0.631579,1.0,0.818966,4,2,2,0,...,0.136364,0.5,-0.170370,-0.200000,-0.155556,0.288889,-0.155556,0.211111,0.155556,2
1,415,11,1041,0.489423,1.0,0.700321,22,3,0,14,...,0.050000,1.0,-0.426268,-1.000000,-0.100000,0.975000,0.300000,0.475000,0.300000,4
2,625,9,486,0.599585,1.0,0.727273,4,3,1,0,...,0.062500,0.7,-0.387821,-1.000000,-0.050000,0.000000,0.000000,0.500000,0.000000,10
3,148,14,505,0.509018,1.0,0.718861,8,4,1,1,...,0.100000,1.0,-0.284722,-0.400000,-0.050000,0.000000,0.000000,0.500000,0.000000,13
4,294,14,274,0.620301,1.0,0.726190,5,1,1,0,...,0.100000,0.6,-0.333333,-0.333333,-0.333333,0.000000,0.000000,0.500000,0.000000,26


In [165]:
test_pred_df = test_df.drop('article_id', axis=1).drop('timedelta', axis=1)
columns_to_standardize = ['n_tokens_title', 'n_tokens_content', 'n_unique_tokens', 'n_non_stop_words',
                            'n_non_stop_unique_tokens', 'num_hrefs', 'num_self_hrefs', 'num_imgs', 'num_videos',
                            'average_token_length', 'num_keywords', 'kw_min_min', 'kw_max_min', 'kw_avg_min',
                            'kw_min_max', 'kw_max_max', 'kw_avg_max', 'kw_min_avg', 'kw_max_avg', 'kw_avg_avg',
                            'self_reference_min_shares', 'self_reference_max_shares', 'self_reference_avg_sharess',
                            'LDA_00', 'LDA_01', 'LDA_02', 'LDA_03', 'LDA_04', 'global_subjectivity',
                            'global_sentiment_polarity', 'global_rate_positive_words', 'global_rate_negative_words',
                            'rate_positive_words', 'rate_negative_words', 'avg_positive_polarity',
                            'min_positive_polarity', 'max_positive_polarity', 'avg_negative_polarity',
                            'min_negative_polarity', 'max_negative_polarity', 'title_subjectivity',
                            'title_sentiment_polarity', 'abs_title_subjectivity', 'abs_title_sentiment_polarity']
scaler = StandardScaler()
test_pred_df[columns_to_standardize] = scaler.fit_transform(test_pred_df[columns_to_standardize])

In [166]:
pred = model.predict(test_pred_df)
pred

310/310 [==============================] - 0s 1ms/step


array([[0.1828552 ],
       [0.2500904 ],
       [0.18938248],
       ...,
       [0.12706983],
       [0.24060792],
       [0.06000388]], dtype=float32)

In [167]:
# Save the predictions
submission = test_df
# drop all columns except article_id which is the last column
submission = submission.drop(submission.columns.difference(['article_id']), axis=1)
submission['score'] = pred
submission

,article_id,score
0,2,0.182855
1,4,0.250090
2,10,0.189382
3,13,0.194907
4,26,0.029879
...,...,...
9906,39630,0.192970
9907,39634,0.056010
9908,39638,0.127070
9909,39640,0.240608


In [169]:
submission.to_csv('../Case studies/News popularity/submission.csv', index=False)